In [1]:
# package_path = '../input/pytorch-image-models/pytorch-image-models-master' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
# import sys; sys.path.append(package_path)

In [16]:
import os
os.environ['TORCH_HOME']= "../"
os.system('apt update')
os.system('apt -y install libgl1-mesa-glx --allow-remove-essential ')

0

In [1]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss
import config
import time

In [2]:
black_bk_path = '/data/fengjing/terr_test/baokong_bk'
# black_bk_path = '/data/fengjing/terr_test/4th_test_5w_white'
model_path = '../checkpoint_efficient_b4/'

In [3]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 224,
    'epochs': 10,
    'train_bs': 200,
    'valid_bs': 1,
    'lr': 1e-4,
    'num_workers': 1,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:1',
#     'device': 'cpu',
    'tta': 3,
    'used_epochs': [6,7,8,9],
    'weights': [1,1,1,1]
}

In [4]:
train = pd.read_csv('baokong_train.csv')
train.head()

,Unnamed: 0,image_id,label
0,0,../data/terrorism_train/bloody_car/bloody_car ...,1
1,1,../data/terrorism_train/bloody_car/bloody_car ...,1
2,2,../data/terrorism_train/bloody_car/bloody_car ...,1
3,3,../data/terrorism_train/bloody_car/bloody_car ...,1
4,4,../data/terrorism_train/bloody_car/bloody_car ...,1


In [5]:
train.label.value_counts()

15    33345
18    11044
20     9940
25     7367
33     5962
14     5614
10     4707
1      3545
29     3307
34     3192
4      3003
30     2309
3      2053
21     1601
6      1600
19     1492
23     1377
5      1361
11     1306
28     1275
8      1265
9      1176
31     1147
17      952
7       933
32      889
22      838
27      756
24      511
2       469
16      417
13      398
12      266
26        1
Name: label, dtype: int64

> We could do stratified validation split in each fold to make each fold's train and validation set looks like the whole train set in target distributions.

In [6]:
submission = pd.read_csv('baokong_test.csv')
submission.head()

,Unnamed: 0,image_id,label
0,0,../data/terrorism_test/baokong_bk/1015_n_v2bf6...,100
1,1,../data/terrorism_test/baokong_bk/1021_132.jpg,100
2,2,../data/terrorism_test/baokong_bk/1022_n_v20f5...,100
3,3,../data/terrorism_test/baokong_bk/1029_n_v274f...,100
4,4,../data/terrorism_test/baokong_bk/1038_n_v24f8...,100


# Helper Functions

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

# img = get_img('/data/fengjing/terr_test/4th_test_5w_white/35634_10.jpg')

# plt.imshow(img)
# plt.show()

# Dataset

In [8]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path =  self.df.iloc[index]['image_id']
#         print(path)
        try:
            img  = get_img(os.path.join(self.data_root,path))
        except:
#             img = get_img('../data/terrorism_test/baokong_bk/992_n_v2801d95e121c64537b14803b01b7b146d.jpg.jpg')
            img = get_img('/data/fengjing/terr_test/4th_test_5w_white/70271_10.jpg')
            print('wrong')
#             print(os.path.join(self.data_root,path))
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

# Define Train\Validation Image Augmentations

In [9]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# Model

In [10]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Main Loop

In [11]:
def post_processing(preds):
    all_bloody_prob = 0
    all_carbloody_prob = 0
    all_normal_prob = 0
    all_flag_prob = 0
    all_emblem_prob = 0
    all_carcrash_prob = 0
    all_uniform_prob = 0
    all_gun_prob = 0
    all_knife_prob = 0
    all_money_prob = 0
    all_politics_pro = 0
    all_scene_prob = 0
    all_drug_prob = 0
    all_map_prob = 0
    for class_info in range(1,len(preds)):
        class_info_ori = class_info
        # if class_info ==15:
        #     print(class_info)
        tmp_name = config.num2name[class_info]
        if tmp_name in config.class_dict2:
            class_info = config.class_dict2[tmp_name]
        score = preds[class_info_ori]

        if class_info==0:
            all_carbloody_prob+=np.round(score*10000)/100
        elif  class_info==1:
            all_carcrash_prob+=np.round(score*10000)/100
        elif class_info==2 or class_info ==3 or class_info ==4:
            all_bloody_prob += np.round(score * 10000) / 100
        elif class_info==5 or class_info ==9 or class_info ==10:
            all_scene_prob += np.round(score * 10000) / 100             #1
        elif class_info == 6 or class_info == 20:
            all_money_prob += np.round(score * 10000) / 100        #1
        elif  class_info==7:
            all_drug_prob+= np.round(score * 10000) / 100      #1
        elif  class_info==13:
            all_map_prob+= np.round(score * 10000) / 100      #1
        elif class_info==12 or class_info ==15 or class_info ==16 or class_info ==18 or class_info ==19 or class_info ==22:
            all_flag_prob += np.round(score * 10000) / 100 #1
        elif class_info==24 or class_info ==26 or class_info ==25 or class_info ==28:
            all_gun_prob += np.round(score * 10000) / 100 #1

        elif  class_info==27:
            all_knife_prob+= np.round(score * 10000) / 100#1
        elif class_info==11 or class_info ==14 or class_info ==17 or class_info ==21:
            all_emblem_prob += np.round(score * 10000) / 100#1

        elif class_info == 8 or class_info == 23:
            all_uniform_prob += np.round(score * 10000) / 100#1
        else:
            all_normal_prob += np.round(score * 10000) / 100

    all_politics_pro = all_scene_prob + all_gun_prob + all_flag_prob + all_emblem_prob + all_uniform_prob + all_knife_prob + all_money_prob + all_drug_prob + all_map_prob

    class_list = [("bloody", all_bloody_prob),
("carbloody", all_carbloody_prob),
("carcarsh", all_carcrash_prob),
("gun", all_gun_prob),
("flag", all_flag_prob),
("emblem", all_emblem_prob),
("uniform", all_uniform_prob),
("knife", all_knife_prob),
("money", all_money_prob),
("normal", all_normal_prob),
("politics", all_politics_pro),
("scene", all_scene_prob),
("drug", all_drug_prob),
("map", all_map_prob),]

    score_list =[all_bloody_prob,
        all_carbloody_prob,
        all_normal_prob,
        all_flag_prob,
        all_emblem_prob,
        all_carcrash_prob,
        all_uniform_prob,
        all_gun_prob,
        all_knife_prob,
        all_money_prob,
        all_politics_pro,
        all_scene_prob,
        all_drug_prob,
        all_map_prob,]
    a = np.array(score_list)
    max_score = np.max(a)
    max_score_index = score_list.index(max_score
                                       )

    return class_list[max_score_index]


In [12]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    recall_num = 0
    recall_list = [0]*9
    wrong_num = 0
    all_num = 0
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        
        preds = torch.softmax(image_preds, 1).detach().cpu().numpy()
        for i in range(preds.shape[0]):
            class_info, score = post_processing(preds[i])
            if score > 90 and 'normal' not in class_info:

                recall_num += 1

            # else:

#                 print(class_info, score)

            if score > 90 and 'normal' not in class_info:

                recall_list[0] += 1
            if score > 85 and 'normal' not in class_info:
                recall_list[1] += 1
            if score > 80 and 'normal' not in class_info:
                recall_list[2] += 1
            if score > 75 and 'normal' not in class_info:
                recall_list[3] += 1
            if score > 70 and 'normal' not in class_info:
                recall_list[4] += 1
            if score > 65 and 'normal' not in class_info:
                recall_list[5] += 1
            if score > 60 and 'normal' not in class_info:
                recall_list[6] += 1
            if score > 55 and 'normal' not in class_info:
                recall_list[7] += 1
            if score > 50 and 'normal' not in class_info:
                recall_list[8] += 1
            all_num+=1
        
        
    print(all_num)
    print(recall_num,all_num)
    print(recall_list,all_num)
    print(np.array(recall_list)*1./all_num)
#     return image_preds_all

In [13]:
if __name__ == '__main__':
    
        seed_everything(CFG['seed'])
        test = pd.DataFrame()
        test['image_id'] = list(os.listdir(black_bk_path))
        test_ds = CassavaDataset(test, black_bk_path, transforms=get_inference_transforms(), output_label=False)
        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        device = torch.device(CFG['device'])
        model = CassvaImgClassifier(CFG['model_arch'], 35).to(device)
        
        val_preds = []
        tst_preds = []
        #for epoch in range(CFG['epochs']-3):
#         for i, epoch in enumerate(CFG['used_epochs']):  
#             model_path_final = os.path.join(model_path,'{}_fold_{}_{}'.format(CFG['model_arch']), fold, epoch)
        model_path_final = os.path.join(model_path,'tf_efficientnet_b4_ns_fold_0_19.pth')

        print(model_path_final)
        checkpoint = torch.load(model_path_final,map_location=device)
#             print(checkpoint.values())
        model.load_state_dict(checkpoint['state_dict'])

#             model.load_state_dict(torch.load(model_path_final,map_location=torch.device('cpu')))

        with torch.no_grad():
#             for _ in range(CFG['tta']):
            a = time.time()
            inference_one_epoch(model, tst_loader, device)
            print('耗时：',time.time()-a)
                    
#                     tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

#         val_preds = np.mean(val_preds, axis=0) 
#         tst_preds = np.mean(tst_preds, axis=0) 
    

../checkpoint_efficient_b4/tf_efficientnet_b4_ns_fold_0_19.pth


  0%|          | 0/250 [00:00<?, ?it/s]

wrong
wrong
wrong
wrong
wrong


  0%|          | 1/250 [00:05<20:47,  5.01s/it]

wrong


...

wrong
wrong
wrong
wrong


100%|█████████▉| 249/250 [23:27<00:06,  6.18s/it]

wrong
wrong
wrong
wrong


100%|██████████| 250/250 [23:33<00:00,  5.66s/it]

50000
894 50000
[894, 1184, 1441, 1685, 1946, 2202, 2493, 2764, 3067] 50000
[0.01788 0.02368 0.02882 0.0337  0.03892 0.04404 0.04986 0.05528 0.06134]
耗时： 1413.8855011463165


In [14]:
a = np.array([894, 1184, 1441, 1685, 1946, 2202, 2493, 2764, 3067])
b = np.array([1837, 1890, 1926, 1959, 1992, 2019, 2046, 2068, 2091] )
c = b/(a+b)

In [15]:
c

array([0.67264738, 0.61483409, 0.57202257, 0.53759605, 0.50584053,
       0.47832267, 0.45076008, 0.42798013, 0.40538969])

In [12]:
cache()

Inference fold 0 started


100%|██████████| 1/1 [00:00<00:00,  7.79it/s]


fold 0 validation loss = 0.21773
fold 0 validation accuracy = 0.93224


In [16]:
torch.cuda.is_available()

False

In [13]:
# test['label'] = np.argmax(tst_preds, axis=1)
# test.head()

,image_id,label
0,2216849948.jpg,4


In [14]:
# test.to_csv('submission.csv', index=False)

# Train part is here: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug